# 因子回测

## 导入模块

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import ticker
from scipy import stats
import feather
import os
import statsmodels.api as sm
import sunlandsdatasdk as sd

## 定义回测区间

In [2]:
start_date = '2019-01-01'
end_date = '2025-01-01'

## 读入和处理数据

### 读入 ST 股票

读入全公司列表

In [3]:
issues = feather.read_dataframe('../data/issues.feather')
issues_tot = issues['issue'].sort_values().unique()

读入 ST 股票

In [4]:
# sd.auth('*', '*')
# st = sd.get_st_issue(issues=list(issues_tot), start_date=start_date, end_date=end_date)
# st = st.set_index('date').stack().reset_index().rename(columns={'level_1': 'issue', 0: 'st'})
# feather.write_dataframe(st, '../data/st.feather')
st = feather.read_dataframe('../data/st.feather')

### 读入日线数据

读入日线数据

In [5]:
price_1d = feather.read_dataframe('../data/StockPriceK1d_20241231.feather')

计算调仓日

In [6]:
trade_date = price_1d['date'].sort_values().unique()
mes = pd.date_range(start=start_date, end=end_date, freq='1ME')
adj_date = np.array([], dtype=np.datetime64)
for me in mes:
    trade_date_before = trade_date[trade_date <= me]
    ad = trade_date_before[-1]
    adj_date = np.append(adj_date, ad)
map_mon_adj = {ad.year * 100 + ad.month: ad for ad in adj_date}

计算每两个调仓日之间的收益

In [7]:
price_1d['year_mon'] = price_1d['date'].dt.year * 100 + price_1d['date'].dt.month

def ret_acc_prod(price_1d):
    ret = price_1d['ret']
    return (1 + ret).prod() - 1

def ret_acc(price_1d):
    preclose = price_1d.iloc[0]['preclose']
    close = price_1d.iloc[-1]['close']
    return (close - preclose) / preclose

def ret_acc_adj(price_1d):
    hfq = price_1d['close'] * price_1d['adj']
    return hfq / hfq.shift(1) - 1

def ret_acc_adj_2(price_1d):
    open_hfq = price_1d.iloc[0]['open'] * price_1d.iloc[0]['adj']
    close_hfq = price_1d.iloc[-1]['close'] * price_1d.iloc[-1]['adj']
    return close_hfq / open_hfq - 1

# price_prod = (
#     price_1d[(price_1d['date'] >= start_date) & (price_1d['date'] <= end_date)]
#         .groupby(['issue', 'year_mon'])[['preclose', 'close', 'adj', 'ret']]
#         .apply(ret_acc_prod)
#         .reset_index()
# )

mon_last = (
    price_1d[(price_1d['date'] >= '2018-12-01') & (price_1d['date'] <= end_date)]
        .groupby(['issue', 'year_mon'])
        .last()
)
price_adj = (
    mon_last
        .groupby('issue', group_keys=False)
        .apply(ret_acc_adj)
        .reset_index()
        .dropna()
)

price_adj = price_adj.rename(columns={0: 'ret'})
price_adj['date'] = price_adj['year_mon'].apply(map_mon_adj.get)
price_adj['ret_next'] = price_adj.groupby('issue')['ret'].shift(-1)
price_adj = price_adj[['issue', 'date', 'ret', 'ret_next']]
price_adj = price_adj.dropna(subset='ret_next')

剔除股票

In [8]:
def newly_listed(date, timedelta='180D'):
    start_date = date.min()
    newly = (date <= start_date + pd.Timedelta(timedelta))
    return newly

price_1d['newly_listed'] = (
    price_1d
        .groupby('issue')['date']
        .transform(newly_listed)
)

price_adj = pd.merge(
    price_adj,
    price_1d[['issue', 'date', 'newly_listed', 'is_limit_sell', 'is_limit_buy']],
    on=['issue', 'date'],
    how='left'
)
price_adj = pd.merge(
    price_adj,
    st,
    on=['issue', 'date'],
    how='left'
)
price_adj['st'] = price_adj['st'].ffill()

price_adj = price_adj[
    (~price_adj['newly_listed']) &
    (~price_adj['is_limit_buy'].astype(bool)) &
    (~price_adj['is_limit_sell'].astype(bool))
]
price_adj = price_adj[~price_adj['st']]
price_adj = price_adj[['issue', 'date', 'ret', 'ret_next']]

C:\Users\admin\AppData\Local\Temp\ipykernel_47744\4021416782.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_adj['st'] = price_adj['st'].ffill()


### 计算基准

中证 500

In [9]:
# sd.auth('*', '*')
# index_bar = sd.get_index_bar('000906', start_date='2018-12-01', end_date=end_date)
# index_bar['date'] = pd.to_datetime(index_bar['date'])
# index_bar = index_bar.drop(columns='_id')

# index_bar['year_mon'] = index_bar['date'].dt.year * 100 + index_bar['date'].dt.month
# index_bar = index_bar.groupby('year_mon').last()
# ret_index = index_bar['close'] / index_bar['close'].shift(1) - 1
# ret_index = ret_index.reset_index().dropna().rename(columns={'close': 'ret'})
# ret_index['date'] = ret_index['year_mon'].apply(map_mon_adj.get)
# ret_index['ret_next'] = ret_index['ret'].shift(-1)
# ret_index = ret_index.dropna()
# ret_index = ret_index.set_index('date')

# os.makedirs('../data/benchmark/', exist_ok=True)
# feather.write_dataframe(ret_index, '../data/benchmark/benchmark_000906.feather')
# ret_index

全样本等权

In [10]:
# ret_sample = (
#     price_adj
#         .groupby('date')[['ret', 'ret_next']].mean()
# )
# os.makedirs('../data/benchmark/', exist_ok=True)
# feather.write_dataframe(ret_sample, '../data/benchmark/benchmark_sample.feather')
# ret_sample

## IC 测试

In [11]:
def IC_calc_once(factor, factor_col, ret_col):
    IC, p = stats.spearmanr(factor[factor_col], factor[ret_col])
    return IC

def IC_calc(factor, factor_col):
    f = factor.copy()
    f = pd.merge(
        f,
        price_adj,
        on=['issue', 'date'],
        how='inner'
    )  
    IC = (
        f
            .groupby('date')[[factor_col, 'ret_next']]
            .apply(IC_calc_once, factor_col=factor_col, ret_col='ret_next')
    )
    return IC

## 分组测试

### 计算分组收益和换手率

In [12]:
def grouped_ret(f, factor_col):
    f_date = f.copy()
    qcut = pd.qcut(
        f_date[factor_col],
        q=10,
        labels=np.arange(1, 11)
    )
    f_date['group'] = qcut
    ret = f_date.groupby('group', observed=False)['ret_next'].mean()
    return ret

def top_weight(f, factor_col):
    f_date = f.copy()
    f_date['group'] = pd.qcut(
        f_date[factor_col],
        q=10,
        labels=np.arange(1, 11)
    )
    top = pd.Series(0., index=issues_tot)
    issues_top = f_date.loc[f_date['group'] == 10, 'issue'].to_list()
    top[issues_top] = 1 / len(issues_top)
    return top

def grouped_calc(factor, factor_col, factor_name=None):
    if factor_name is None:
        factor_name = factor_col
    f = pd.merge(
        factor,
        price_adj,
        on=['issue', 'date'],
        how='inner'
    )
    ret = (
        f
            .groupby('date')[[factor_col, 'ret', 'ret_next']]
            .apply(grouped_ret, factor_col=factor_col)
    )
    top = (
        f
            .groupby('date')[[factor_col, 'issue']]
            .apply(top_weight, factor_col=factor_col)
    )
    turnover = np.abs(top - top.shift(1)).sum(axis=1)
    return ret, turnover

### 画分组收益图和多空收益曲线

In [13]:
def grouped_plot(ret, ret_excess, factor_name, dirname=None):
    ret_annual = (1 + ret.mean()) ** 12 - 1
    # ret_annual = (1 + ret).prod() ** (12 / len(ret)) - 1
    plt.bar(ret_annual.index, ret_annual, color='#a60021')
    plt.xlabel('Group')
    plt.ylabel('Annualized Return')
    plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))
    plt.title(factor_name + ' Annualized Return')
    if dirname is None:
        plt.show()
    else:
        os.makedirs(dirname, exist_ok=True)
        plt.savefig(dirname + '/grouped_' + factor_name + '.png')
        plt.close()
    
    long = (1 + ret[10]).cumprod()
    long_excess = (1 + ret_excess[10]).cumprod()
    long_short = (1 + ret[10] - ret[1]).cumprod()
    plt.plot(long, linewidth=3, color='#a60021', label='long')
    plt.plot(long_short, linewidth=3, color='#ffc000', label='long-short')
    ylim = plt.ylim()
    plt.fill_between(
        long_excess.index, 0, long_excess,
        color='#a5a5a5', label='long_excess',
    )
    plt.ylim(ylim)
    plt.legend()
    plt.grid()
    plt.xlabel('Time')
    plt.ylabel('Net Value')
    plt.title(factor_name + ' Grouping Test')
    if dirname is None:
        plt.show()
    else:
        os.makedirs(dirname, exist_ok=True)
        plt.savefig(dirname + '/nv_' + factor_name + '.png')
        plt.close()

### 计算最大回撤

In [14]:
def max_drawdown_calc(ret):
    cumret = (1 + ret).cumprod()
    running_max = cumret.cummax()
    drawdown = cumret / running_max - 1
    return drawdown.min()

## 一键回测

### 单个回测

In [15]:
def backtest(filename, factor_col, factor_name, benchmark, dirname=None):
    if factor_name is None:
        factor_name = factor_col
    factor = feather.read_dataframe(filename)

    df_IC = pd.DataFrame(columns=['factor', 'IC_mean', 'winning', 'IC_IR', 't', 'max', 'min'])
    IC = IC_calc(factor, factor_col=factor_col)

    sign = np.sign(IC.mean())
    factor[factor_col] *= sign

    winning = (np.sign(IC) == sign).sum() / len(IC)
    df_IC.loc[0] = [
        factor_name,
        IC.mean(),
        winning,
        IC.mean() / IC.std(),
        np.sqrt(len(IC)) * IC.mean() / IC.std(),
        IC.max(),
        IC.min()
    ]

    df_grouped = pd.DataFrame(columns=[
        'factor',
        'long_annual', 'excess_annual', 'long_sharpe',
        'long_short',
        'max_drawdown', 'turnover_annual'
    ])
    grouped, turnover = grouped_calc(
        factor, factor_col=factor_col,
        factor_name=factor_name
    )
    if benchmark != None:
        ret_benchmark = feather.read_dataframe(benchmark)
    ret_benchmark = ret_benchmark.reindex(grouped.index)
    excess = grouped.sub(ret_benchmark['ret_next'], axis=0)

    grouped_plot(grouped, excess, factor_name=factor_name, dirname=dirname)

    long_annual = (1 + grouped[10].mean()) ** 12 - 1
    excess_annual = (1 + excess[10].mean()) ** 12 - 1
    std_annual = np.sqrt(12) * excess[10].std()
    long_short = (1 + (grouped[10] - grouped[1]).mean()) ** 12 - 1
    max_drawdown = max_drawdown_calc(grouped[10] - grouped[1])
    turnover_annual = turnover.mean() * 12
    df_grouped.loc[0]=[
        factor_name,
        long_annual,
        excess_annual,
        excess_annual / std_annual,
        long_short,
        max_drawdown,
        turnover_annual
    ]
    
    return df_IC, df_grouped

### 一组因子回测

In [16]:
def set_backtest(filenames, factor_cols, factor_names, benchmark, dirname=None):
    if factor_names is None:
        factor_names = factor_cols
    IC = None
    grouped = None
    for filename, factor_col, factor_name in zip(filenames, factor_cols, factor_names):
        IC_factor, grouped_factor = backtest(
            filename=filename,
            factor_col=factor_col, factor_name=factor_name,
            benchmark=benchmark, dirname=dirname
        )
        IC = pd.concat([IC, IC_factor])
        grouped = pd.concat([grouped, grouped_factor])
    display(IC)
    display(grouped)

## 跳跃收益因子回测

In [17]:
IC, grouped = backtest(
    filename = '../data/ret_jump/neutral_ret_jump.feather',
    factor_col='neutral_factor', factor_name='ret_jump_neutral',
    benchmark='../data/benchmark/benchmark_sample.feather',
    dirname='../data/ret_jump/backtest1/'
)
display(IC)
display(grouped)

,factor,IC_mean,winning,IC_IR,t,max,min
0,ret_jump_neutral,-0.045763,0.815385,-0.900023,-7.256217,0.080992,-0.175871


,factor,long_annual,excess_annual,long_sharpe,long_short,max_drawdown,turnover_annual
0,ret_jump_neutral,0.129016,0.016403,0.399035,0.159508,-0.062559,20.46685


## 点度中心性回测

### N_connect_0_1

In [18]:
cores = np.array(['_num', '_size', '', '_residual'])
filenames = '../data/N_connect/N_connect_0_1/neutral_N_connect' + cores + '.feather'
factor_cols = ['neutral_factor'] * len(cores)
factor_names = 'N_connect' + cores
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906.feather',
    dirname='../data/N_connect/N_connect_0_1/backtest1/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,0.033435,0.828125,0.682919,5.463351,0.135539,-0.121017
0,N_connect_size,0.035143,0.828125,0.694828,5.558625,0.146744,-0.118299
0,N_connect,0.037101,0.828125,0.708104,5.664832,0.143730,-0.126767
0,N_connect_residual,0.012196,0.625000,0.238625,1.909000,0.114039,-0.125696


,factor,long_annual,excess_annual,long_sharpe,long_short,max_drawdown,turnover_annual
0,N_connect_num,0.137631,0.106164,0.743652,0.065244,-0.061241,11.782436
0,N_connect_size,0.146812,0.115112,0.807340,0.064522,-0.090096,9.081106
0,N_connect,0.142911,0.111310,0.776096,0.076917,-0.067847,9.977303
0,N_connect_residual,0.112151,0.081332,0.568994,-0.006778,-0.160994,10.673676


### N_connect_1_any

In [19]:
cores = np.array(['_num', '_size', ''])
filenames = '../data/N_connect/N_connect_1_any/neutral_N_connect' + cores + '.feather'
factor_cols = ['neutral_factor'] * 3
factor_names = 'N_connect' + cores
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906.feather',
    dirname='../data/N_connect/N_connect_1_any/backtest1/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,-0.035074,0.625000,-0.446364,-3.570912,0.121009,-0.224008
0,N_connect_size,-0.030425,0.625000,-0.386713,-3.093705,0.133171,-0.199034
0,N_connect,-0.033345,0.609375,-0.417358,-3.338866,0.128456,-0.211188


,factor,long_annual,excess_annual,long_sharpe,long_short,max_drawdown,turnover_annual
0,N_connect_num,0.156928,0.124972,0.957801,0.096345,-0.056050,9.569343
0,N_connect_size,0.143376,0.111763,0.874273,0.074880,-0.061511,7.894108
0,N_connect,0.152188,0.120352,0.915278,0.089031,-0.050004,8.421827


## 跳跃关联动量因子回测

### 总收益

In [20]:
cores = np.array(['_num', '_size'])
filenames = '../data/peer_ret/neutral_peer' + cores + '.feather'
factor_cols = ['neutral_factor'] * 2
factor_names = 'peer_ret' + cores
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906.feather',
    dirname='../data/N_connect/peer_ret/backtest1/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,peer_ret_num,-0.003228,0.484375,-0.052225,-0.417802,0.153198,-0.139764
0,peer_ret_size,-0.014660,0.593750,-0.263077,-2.104612,0.118527,-0.141054


,factor,long_annual,excess_annual,long_sharpe,long_short,max_drawdown,turnover_annual
0,peer_ret_num,0.071956,0.042161,0.319264,-0.035541,-0.283611,20.001041
0,peer_ret_size,0.089516,0.059273,0.448648,-0.012323,-0.231775,20.183791


### 正 / 负 / 非跳跃收益

In [21]:
cores = np.array([
    'posjump_num', 'posjump_size',
    'negjump_num', 'negjump_size',
    'nojump_num', 'nojump_size'
])
filenames = '../data/peer_ret/neutral_peer_' + cores + '.feather'
factor_names = 'relative_' + cores
factor_cols = ['neutral_factor'] * 6
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906.feather',
    dirname='../data/peer_ret/backtest1/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,relative_posjump_num,-0.048801,0.750000,-0.747943,-5.983543,0.069749,-0.200988
0,relative_posjump_size,-0.051969,0.828125,-0.813302,-6.506420,0.082908,-0.194396
0,relative_negjump_num,0.050284,0.750000,0.787238,6.297904,0.180442,-0.104413
0,relative_negjump_size,0.049394,0.828125,0.751963,6.015705,0.201159,-0.122481
0,relative_nojump_num,0.037674,0.765625,0.570353,4.562827,0.220922,-0.148096
0,relative_nojump_size,0.033274,0.718750,0.540888,4.327105,0.201426,-0.174770


,factor,long_annual,excess_annual,long_sharpe,long_short,max_drawdown,turnover_annual
0,relative_posjump_num,0.144275,0.112640,0.900552,0.081905,-0.072540,13.154160
0,relative_posjump_size,0.139883,0.108359,0.845810,0.079098,-0.095428,13.242679
0,relative_negjump_num,0.158962,0.126955,0.964512,0.098706,-0.073529,13.113915
0,relative_negjump_size,0.137521,0.106058,0.817843,0.092101,-0.071338,12.737970
0,relative_nojump_num,0.162999,0.130889,0.920377,0.130282,-0.091282,17.960634
0,relative_nojump_size,0.149558,0.117789,0.811579,0.103416,-0.114100,18.432234


### 非正跳跃收益

In [22]:
# cores = np.array([
    'without_posjump_num', 'without_posjump_size',
    'without_posjump', 'without_posjump_residual'
])
filenames = '../data/peer_ret/neutral_peer_' + cores + '.feather'
factor_names = 'relative_' + cores
factor_cols = ['neutral_factor'] * len(cores)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906.feather',
    dirname='../data/peer_ret/backtest1/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,relative_without_posjump_num,0.053846,0.781250,0.798114,6.384915,0.228976,-0.128277
0,relative_without_posjump_size,0.050861,0.859375,0.782017,6.256139,0.220595,-0.148684
0,relative_without_posjump,0.055513,0.796875,0.805777,6.446220,0.235157,-0.146688
0,relative_without_posjump_residual,0.017892,0.609375,0.306461,2.451686,0.154696,-0.124087


,factor,long_annual,excess_annual,long_sharpe,long_short,max_drawdown,turnover_annual
0,relative_without_posjump_num,0.173024,0.140660,1.019740,0.141165,-0.081640,13.979588
0,relative_without_posjump_size,0.160418,0.128373,0.944120,0.144267,-0.078296,14.367149
0,relative_without_posjump,0.167902,0.135668,0.986077,0.150955,-0.084130,13.968014
0,relative_without_posjump_residual,0.132925,0.101578,0.707178,0.050706,-0.108854,15.522003
